In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
import pandas as pd

df = pd.read_csv('/kaggle/input/renmindaily/RenMin_Daily.csv')
df

,id,date,title,content
0,20210101-01-01,20210101,中俄两国元首互致新年贺电 中俄两国总理互致新年贺电,新华社北京12月31日电 2020年12月31日，国家主席习近平和俄罗斯总统普京互致新年...
1,20210101-01-02,20210101,国家主席习近平发表二〇二一年新年贺词,■ 2020年是极不平凡的一年。面对突如其来的新冠肺炎疫情，我们以人民至上、生命至上诠释...
2,20210101-01-03,20210101,艰难方显勇毅，磨砺始得玉成 ——习近平主席二〇二一年新年贺词启示录①,新故相推，日生不滞。在风雨兼程中，我们告别2020年，迎来2021年。\n “艰难方显...
3,20210101-01-04,20210101,全国政协举行新年茶话会 习近平发表重要讲话李克强栗战书王沪宁赵乐际韩正王岐山出席 汪洋主持,新华社北京12月31日电 中国人民政治协商会议全国委员会12月31日上午在全国政协礼堂举...
4,20210101-02-01,20210101,在全国政协新年茶话会上的讲话 （2020年12月31日）,同志们，朋友们：\n 在风雨兼程中，我们即将送别2020年，迎来2021年新年。很高兴在这...
...,...,...,...,...
106329,20250517-08-02,20250517,数字化，让文物迸发丰沛的生命力,\t\t\t\t\t\t\t许 蒙\n\t\t\t\t\t\t\t\t《人民日报》（202...
106330,20250517-08-03,20250517,看中国也看世界,\t\t\t\t\t\t\t王 樾\n\t\t\t\t\t\t\t\t《人民日报》（202...
106331,20250517-08-04,20250517,小县城有所“大学校”,\t\t\t\t\t\t\t邓建胜\n\t\t\t\t\t\t\t\t《人民日报》（2025...
106332,20250517-08-05,20250517,文创，萃取文物的美学价值,\t\t\t\t\t\t\t胡继禹\n\t\t\t\t\t\t\t\t《人民日报》（2025...


In [2]:
import csv

def csv_to_text(csv_path, txt_path, separator=' '):
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        all_text = []
        for row in reader:
            # 将每行的所有单元格用分隔符连接
            row_text = separator.join(str(cell) for cell in row)
            all_text.append(row_text)
        
        # 将所有行合并为一个字符串
        final_text = '\n'.join(all_text)
        
        # 写入文本文件
        with open(txt_path, 'w', encoding='utf-8') as txtfile:
            txtfile.write(final_text)

# 使用示例
csv_to_text('/kaggle/input/renmindaily/RenMin_Daily.csv', 'output.txt', separator=' ')
print("保存成功")

保存成功


In [ ]:
with open('/kaggle/working/output.txt') as f:
    text = f.read()
text

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.attention import SDPBackend, sdpa_kernel

def get_batch(split):
    # 选择训练或验证数据集
    data = train_data if split == 'train' else val_data

    # 动态从数据集中选择位置索引
    ix = torch.randint(len(data) - block_size, (batch_size,)) # [0,103846]随机生成位置索引，向后截取block_size字符训练
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x.to(device),y.to(device)

class Block(nn.Module):

    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, n_embd, head_size, dropout)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))   # 残差连接
        x = x + self.ffwd(self.ln2(x)) # 残差连接
        return x

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.ReLU(),
            nn.Dropout(0.1) # 增加正则化防止过拟合
        )

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, n_embd, head_embd, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head_embd, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, n_embd, head_embd, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_embd, bias=False)
        self.query = nn.Linear(n_embd, head_embd, bias=False)
        self.value = nn.Linear(n_embd, head_embd, bias=False)
        # self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_x):
        B, T, C = input_x.shape
        k = self.key(input_x)   # (B, T, head_size)
        q = self.query(input_x) # (B, T, head_size)
        v = self.value(input_x) # (B, T, 16)

        # wei = q @ k.transpose(-2,-1) * C ** -0.5

        # T = wei.shape[-1]
        # tril = torch.tril(torch.ones(T, T, device=wei.device))
        # wei = wei.masked_fill(tril == 0, float('-inf'))
        # wei = wei.softmax(dim=-1)
        # wei = self.dropout(wei)
        
        # out = wei @ v
        # return out

         ##############################################
        # 关键修改：将3维转换为4维，适配优化内核要求
        # 单个头的num_heads=1，因此维度变为 [B, 1, T, head_size]
        ##############################################
        q = q.view(B, T, 1, -1).transpose(1, 2)  # [B, 1, T, head_size]
        k = k.view(B, T, 1, -1).transpose(1, 2)  # [B, 1, T, head_size]
        v = v.view(B, T, 1, -1).transpose(1, 2)  # [B, 1, T, head_size]

        # 移除强制指定后端的逻辑，让PyTorch自动选择可用内核
        # 注意：若环境不支持Flash Attention，会自动使用基础内核
        attn_output = F.scaled_dot_product_attention(
            q, k, v, 
            is_causal=True  # 因果掩码（适用于语言模型）
        )  # 输出：[B, 1, T, head_size]

        ##############################################
        # 将4维转换回3维，方便后续拼接
        ##############################################
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, -1)  # [B, T, head_size]
        
        return attn_output


class BingramLanguageModel(nn.Module):
    
    def __init__(self, block_size, vocab_size, n_embd, n_head, n_layer, dropout):
        super().__init__()
        # 每个token直接输出的logits值作为下一个token的映射
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, dropout=dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx和target都是维度为 (B,T) 的整型tensor
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device), ) # (T,C)
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B * T, C)
            targets = targets.reshape(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx指当前语料集(B,T)中的索引
        for _ in range(max_new_tokens):
            # 限定索引列的取值范围
            idx_cond = idx[:, -block_size:]
            # 推理
            logits, loss = self(idx_cond)
            # 只提取最后一个时间步的结果
            logits = logits[:, -1, :]  # (B,C)
            # 通过softmax转换为概率值
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # 随机采样
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # 把采样的索引追加在当前解码序列末尾
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

if __name__ == '__main__':

    # 模型训练数据集
    block_size = 8
    batch_size = 32
    max_iter = 10000
    learn_rate = 1e-3
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    n_embd = 64
    eval_interval = 500
    eval_iters = 200
    head_size = 4
    num_layers = 4
    dropout = 0.1

    
    with open('/kaggle/working/output.txt') as f:
        text = f.read()

    # 字典、编码器(函数)、解码器(函数)
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = {ch:i for i,ch in enumerate(chars)}  #str_to_index
    itos = {i:ch for i,ch in enumerate(chars)}  #index_to_str

    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])

    # 文本转换token index
    data = torch.tensor(encode(text), dtype=torch.long)

    # 拆分数据集
    n = int(len(data) * .9)
    train_data = data[:n]
    val_data = data[n:]

    # 模型训练
    model = BingramLanguageModel(block_size, vocab_size, n_embd, head_size, num_layers, dropout)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate)

    @torch.no_grad()
    def estimate_loss():
        out = {}
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        model.train()
        return out

    for iter in range(max_iter):

        if iter % eval_interval == 0:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # 批次样本
        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # 模型生成
    idx = torch.zeros((1,1), dtype=torch.long, device=device)
    print(decode(model.generate(idx, max_new_tokens=500)[0].tolist())) 

step 0: train loss 9.1156, val loss 9.1033
step 500: train loss 5.7105, val loss 5.9007
step 1000: train loss 5.2155, val loss 5.4039
step 1500: train loss 5.0613, val loss 5.2635
step 2000: train loss 4.8921, val loss 5.0986
step 2500: train loss 4.8114, val loss 5.0295
step 3000: train loss 4.7597, val loss 4.9371
step 3500: train loss 4.7312, val loss 4.9312
step 4000: train loss 4.6638, val loss 4.8364
step 4500: train loss 4.6868, val loss 4.7963
step 5000: train loss 4.5959, val loss 4.7820
step 5500: train loss 4.5611, val loss 4.7523
step 6000: train loss 4.5659, val loss 4.7316
step 6500: train loss 4.5222, val loss 4.6934
step 7000: train loss 4.4912, val loss 4.6524
step 7500: train loss 4.4890, val loss 4.6605
step 8000: train loss 4.4522, val loss 4.6328
step 8500: train loss 4.4396, val loss 4.6332
step 9000: train loss 4.4424, val loss 4.6608
step 9500: train loss 4.4129, val loss 4.5981
			生台。
　　“先后，中国助联合建民生的中方的精神的来砥砺，坚持有力培育“双人、墅圈评汀首先树隔离节继文化部级、企业1年10月，古米直来、王天中凡所献拳身鹿瞧的任务